<a href="https://colab.research.google.com/github/nandakumar92/sql_queries_to_pandas/blob/main/formula_one_races_sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import pandas as pd
import numpy as np

In [3]:
circuitsdf=pd.read_csv('/content/gdrive/MyDrive/Colab/Data/formula_1_races/circuits.csv')
driversdf=pd.read_csv('/content/gdrive/MyDrive/Colab/Data/formula_1_races/drivers.csv')
racesdf=pd.read_csv('/content/gdrive/MyDrive/Colab/Data/formula_1_races/races.csv')
resultsdf=pd.read_csv('/content/gdrive/MyDrive/Colab/Data/formula_1_races/results.csv')
statusdf=pd.read_csv('/content/gdrive/MyDrive/Colab/Data/formula_1_races/status.csv')
pitstopsdf=pd.read_csv('/content/gdrive/MyDrive/Colab/Data/formula_1_races/pit_stops.csv')
laptimesdf=pd.read_csv('/content/gdrive/MyDrive/Colab/Data/formula_1_races/lap_times.csv')

In [4]:
circuitsdf.head()

,circuitId,circuitRef,name,location,country,lat,lng,alt,url
0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.96800,10,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
1,2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.73800,\N,http://en.wikipedia.org/wiki/Sepang_Internatio...
2,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.03250,50.51060,\N,http://en.wikipedia.org/wiki/Bahrain_Internati...
3,4,catalunya,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.57000,2.26111,\N,http://en.wikipedia.org/wiki/Circuit_de_Barcel...
4,5,istanbul,Istanbul Park,Istanbul,Turkey,40.95170,29.40500,\N,http://en.wikipedia.org/wiki/Istanbul_Park


In [6]:
driversdf.head()

,driverId,driverRef,number,code,forename,surname,dob,nationality,url
0,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton
1,2,heidfeld,\N,HEI,Nick,Heidfeld,1977-05-10,German,http://en.wikipedia.org/wiki/Nick_Heidfeld
2,3,rosberg,6,ROS,Nico,Rosberg,1985-06-27,German,http://en.wikipedia.org/wiki/Nico_Rosberg
3,4,alonso,14,ALO,Fernando,Alonso,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso
4,5,kovalainen,\N,KOV,Heikki,Kovalainen,1981-10-19,Finnish,http://en.wikipedia.org/wiki/Heikki_Kovalainen


In [7]:
racesdf.head()

,raceId,year,round,circuitId,name,date,time,url
0,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...
1,2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...
2,3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Gran...
3,4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...
4,5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Gran...


In [8]:
resultsdf.head()

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1
1,2,18,2,2,3,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1
2,3,18,3,3,7,7,3,3,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1
3,4,18,4,4,5,11,4,4,4,5.0,58,+17.181,5707797,58,7,1:28.603,215.464,1
4,5,18,5,1,23,3,5,5,5,4.0,58,+18.014,5708630,43,1,1:27.418,218.385,1


In [9]:
statusdf.head()

,statusId,status
0,1,Finished
1,2,Disqualified
2,3,Accident
3,4,Collision
4,5,Engine


In [10]:
pitstopsdf.head()

,raceId,driverId,stop,lap,time,duration,milliseconds
0,841,153,1,1,17:05:23,26.898,26898
1,841,30,1,1,17:05:52,25.021,25021
2,841,17,1,11,17:20:48,23.426,23426
3,841,4,1,12,17:22:34,23.251,23251
4,841,13,1,13,17:24:10,23.842,23842


In [11]:
laptimesdf.head()

,raceId,driverId,lap,position,time,milliseconds
0,841,20,1,1,1:38.109,98109
1,841,20,2,1,1:33.006,93006
2,841,20,3,1,1:32.713,92713
3,841,20,4,1,1:32.803,92803
4,841,20,5,1,1:32.342,92342


In [76]:
tempdf=resultsdf.merge(driversdf,how='left',on='driverId').merge(racesdf,how='left',on='raceId').merge(circuitsdf,how='left',on='circuitId')\
.merge(statusdf,how='left',on='statusId')
tempdf.head(2)

,resultId,raceId,driverId,constructorId,number_x,grid,position,positionText,positionOrder,points,laps,time_x,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId,driverRef,number_y,code,forename,surname,dob,nationality,url_x,year,round,circuitId,name_x,date,time_y,url_y,circuitRef,name_y,location,country,lat,lng,alt,url,status
0,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,2008,1,1,Australian Grand Prix,2008-03-16,04:30:00,http://en.wikipedia.org/wiki/2008_Australian_G...,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,http://en.wikipedia.org/wiki/Melbourne_Grand_P...,Finished
1,2,18,2,2,3,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1,heidfeld,\N,HEI,Nick,Heidfeld,1977-05-10,German,http://en.wikipedia.org/wiki/Nick_Heidfeld,2008,1,1,Australian Grand Prix,2008-03-16,04:30:00,http://en.wikipedia.org/wiki/2008_Australian_G...,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,http://en.wikipedia.org/wiki/Melbourne_Grand_P...,Finished


In [56]:
'''select
year,
driverRef,
sum(points)
from 
Data 
'''
tempdf.groupby(['year','driverRef'],as_index=False).agg({'points':'sum'})\
.sort_values(by=['year','driverRef'],ascending=[False,True])



,year,driverRef,points
3099,2020,aitken,0.0
3100,2020,albon,105.0
3101,2020,bottas,223.0
3102,2020,gasly,75.0
3103,2020,giovinazzi,4.0
...,...,...,...
76,1950,villoresi,0.0
77,1950,wallard,0.0
78,1950,walt_brown,0.0
79,1950,webb,0.0


In [75]:
""" select
year,
driveref,
sum(points) as points,
sum()over(partition by driverref order by year asc) as cum_points
from
Data
"""

df2=tempdf.groupby(['year','driverRef'],as_index=False).agg({'points':'sum'})
df2['cum_points']=df2.groupby(['driverRef']).points.transform('cumsum')
df2[df2.driverRef=='ascari']


,year,driverRef,points,cum_points
2,1950,ascari,11.0,11.0
84,1951,ascari,28.0,39.0
168,1952,ascari,53.5,92.5
274,1953,ascari,46.5,139.0
382,1954,ascari,1.0,140.0
477,1955,ascari,0.0,140.0


In [35]:
### select * from results where raceId=10 order by points desc limit 10

resultsdf[resultsdf.raceId==10].nlargest(10,columns='points')

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
7733,7734,10,1,1,1,4,1,1,1,10.0,70,1:38:23.876,5903876,16,4,1:22.479,191.219,1
7734,7735,10,8,6,4,7,2,2,2,8.0,70,+11.529,5915405,70,2,1:22.434,191.323,1
7735,7736,10,17,9,14,3,3,3,3,6.0,70,+16.886,5920762,65,1,1:21.931,192.498,1
7736,7737,10,3,3,16,5,4,4,4,5.0,70,+26.967,5930843,65,3,1:22.468,191.245,1
7737,7738,10,5,1,2,6,5,5,5,4.0,70,+34.392,5938268,63,8,1:22.958,190.115,1
7738,7739,10,10,7,10,13,6,6,6,3.0,70,+35.237,5939113,63,5,1:22.506,191.157,1
7739,7740,10,18,23,22,8,7,7,7,2.0,70,+55.088,5958964,57,6,1:22.706,190.694,1
7740,7741,10,15,7,9,11,8,8,8,1.0,70,+1:08.172,5972048,65,13,1:23.261,189.423,1
7741,7742,10,6,3,17,9,9,9,9,0.0,70,+1:08.774,5972650,55,11,1:23.180,189.608,1
7742,7743,10,22,23,23,12,10,10,10,0.0,70,+1:09.256,5973132,55,9,1:23.024,189.964,1


In [42]:
### select * from results where raceId=10 order by time asc limit 10

resultsdf[resultsdf.raceId==10].sort_values(by=['time'],ascending=True).head(10)

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
7734,7735,10,8,6,4,7,2,2,2,8.0,70,+11.529,5915405,70,2,1:22.434,191.323,1
7735,7736,10,17,9,14,3,3,3,3,6.0,70,+16.886,5920762,65,1,1:21.931,192.498,1
7740,7741,10,15,7,9,11,8,8,8,1.0,70,+1:08.172,5972048,65,13,1:23.261,189.423,1
7741,7742,10,6,3,17,9,9,9,9,0.0,70,+1:08.774,5972650,55,11,1:23.180,189.608,1
7742,7743,10,22,23,23,12,10,10,10,0.0,70,+1:09.256,5973132,55,9,1:23.024,189.964,1
7743,7744,10,2,2,6,15,11,11,11,0.0,70,+1:10.612,5974488,65,14,1:23.282,189.375,1
7744,7745,10,12,4,8,14,12,12,12,0.0,70,+1:11.512,5975388,65,15,1:23.418,189.067,1
7745,7746,10,9,2,5,18,13,13,13,0.0,70,+1:14.046,5977922,65,12,1:23.224,189.507,1
7736,7737,10,3,3,16,5,4,4,4,5.0,70,+26.967,5930843,65,3,1:22.468,191.245,1
7737,7738,10,5,1,2,6,5,5,5,4.0,70,+34.392,5938268,63,8,1:22.958,190.115,1
